# Módulo 8: Vision Transformers e Mecanismos de Atenção## Objetivos de Aprendizagem- Compreender os fundamentos dos mecanismos de atenção- Entender a arquitetura Vision Transformer (ViT)- Implementar modelos de atenção para visão computacional- Analisar vantagens e limitações dos Vision Transformers- Aplicar ViT em tarefas práticas de visão computacional---## 8.1 Introdução aos Mecanismos de Atenção**Mecanismos de Atenção** são componentes fundamentais que permitem aos modelos focar em partes específicas dos dados de entrada, revolucionando tanto o processamento de linguagem natural quanto a visão computacional.![Introdução Mecanismos Atenção](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/introducao_mecanismos_atencao.png)### Conceitos Fundamentais**Definição de Atenção:**- **Foco seletivo**: Capacidade de concentrar recursos computacionais em partes relevantes- **Pesos dinâmicos**: Atribuição de importância variável a diferentes elementos- **Contexto global**: Consideração de todas as informações disponíveis- **Processamento paralelo**: Capacidade de processar múltiplas relações simultaneamente**Analogia Biológica:**- **Atenção humana**: Foco seletivo em estímulos relevantes- **Processamento visual**: Concentração em características importantes- **Memória de trabalho**: Manutenção de informações relevantes- **Decisão**: Integração de múltiplas fontes de informação### Tipos de Atenção**1. Self-Attention (Atenção Própria):**- **Relações internas**: Cada elemento interage com todos os outros- **Pesos computados**: Importância calculada dinamicamente- **Contexto completo**: Consideração de toda a sequência- **Paralelização**: Processamento simultâneo**2. Multi-Head Attention:**- **Múltiplas cabeças**: Diferentes representações de atenção- **Diversidade**: Captura diferentes tipos de relações- **Concatenação**: Combinação de múltiplas cabeças- **Flexibilidade**: Maior capacidade expressiva**3. Cross-Attention:**- **Interação**: Entre sequências diferentes- **Query**: De uma sequência- **Key/Value**: De outra sequência- **Aplicação**: Tradução, geração, multimodal![Tipos de Atenção](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/tipos_atencao.png)### Evolução dos Mecanismos de Atenção**Progressão Histórica:**- **2014**: Attention mechanism introduzido para tradução- **2017**: Transformer com self-attention- **2018**: BERT com atenção bidirecional- **2020**: Vision Transformer (ViT) para imagens- **2021**: CLIP com atenção multimodal- **2022**: DALL-E com atenção para geração![Evolução Mecanismos Atenção](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/evolucao_mecanismos_atencao.png)**Marcos Importantes:**- **2014**: Neural Machine Translation by Jointly Learning to Align and Translate - Bahdanau et al.- **2017**: Attention Is All You Need - Vaswani et al.- **2020**: An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.- **2021**: CLIP: Learning Transferable Visual Representations - Radford et al.**Referências:**- [Attention Is All You Need - Vaswani et al.](https://arxiv.org/abs/1706.03762)- [An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.](https://arxiv.org/abs/2010.11929)

## 8.2 Demonstração Prática: Mecanismos de Atenção

Vamos implementar e visualizar diferentes tipos de atenção:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

class AttentionMechanismsDemo:
    """Demonstração de diferentes mecanismos de atenção"""
    
    def __init__(self, seq_len=10, d_model=64):
        self.seq_len = seq_len
        self.d_model = d_model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def create_sample_data(self):
        """Cria dados de exemplo para demonstração"""
        
        # Criar sequência de entrada
        x = torch.randn(1, self.seq_len, self.d_model)
        
        # Adicionar padrões específicos
        # Padrão 1: Valores altos no meio
        x[0, 3:7, :] += 2.0
        
        # Padrão 2: Valores baixos no início
        x[0, 0:2, :] -= 1.5
        
        # Padrão 3: Valores médios no final
        x[0, 8:, :] += 0.5
        
        return x
    
    def self_attention(self, x):
        """Implementa self-attention"""
        
        # Linear transformations
        W_q = nn.Linear(self.d_model, self.d_model, bias=False)
        W_k = nn.Linear(self.d_model, self.d_model, bias=False)
        W_v = nn.Linear(self.d_model, self.d_model, bias=False)
        
        # Compute Q, K, V
        Q = W_q(x)
        K = W_k(x)
        V = W_v(x)
        
        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_model)
        attention_weights = F.softmax(scores, dim=-1)
        
        # Apply attention to values
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def multi_head_attention(self, x, num_heads=8):
        """Implementa multi-head attention"""
        
        class MultiHeadAttention(nn.Module):
            def __init__(self, d_model, num_heads):
                super(MultiHeadAttention, self).__init__()
                self.d_model = d_model
                self.num_heads = num_heads
                self.d_k = d_model // num_heads
                
                self.W_q = nn.Linear(d_model, d_model)
                self.W_k = nn.Linear(d_model, d_model)
                self.W_v = nn.Linear(d_model, d_model)
                self.W_o = nn.Linear(d_model, d_model)
                
            def forward(self, x):
                batch_size, seq_len, d_model = x.size()
                
                # Linear transformations
                Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
                K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
                V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
                
                # Compute attention scores
                scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_k)
                attention_weights = F.softmax(scores, dim=-1)
                
                # Apply attention to values
                attended_values = torch.matmul(attention_weights, V)
                
                # Concatenate heads
                attended_values = attended_values.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
                
                # Final linear transformation
                output = self.W_o(attended_values)
                
                return output, attention_weights
        
        mha = MultiHeadAttention(self.d_model, num_heads)
        output, attention_weights = mha(x)
        
        return output, attention_weights
    
    def cross_attention(self, x, context):
        """Implementa cross-attention"""
        
        # Linear transformations
        W_q = nn.Linear(self.d_model, self.d_model, bias=False)
        W_k = nn.Linear(self.d_model, self.d_model, bias=False)
        W_v = nn.Linear(self.d_model, self.d_model, bias=False)
        
        # Compute Q from x, K and V from context
        Q = W_q(x)
        K = W_k(context)
        V = W_v(context)
        
        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_model)
        attention_weights = F.softmax(scores, dim=-1)
        
        # Apply attention to values
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def visualize_attention_weights(self, attention_weights, title="Attention Weights"):
        """Visualiza os pesos de atenção"""
        
        # Converter para numpy
        if isinstance(attention_weights, torch.Tensor):
            attention_weights = attention_weights.detach().numpy()
        
        # Se for multi-head, usar a média
        if len(attention_weights.shape) == 4:  # (batch, heads, seq, seq)
            attention_weights = np.mean(attention_weights, axis=1)
        
        # Remover dimensão do batch se existir
        if len(attention_weights.shape) == 3:
            attention_weights = attention_weights[0]
        
        # Criar heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(attention_weights, 
                    annot=True, 
                    fmt='.2f', 
                    cmap='Blues',
                    xticklabels=[f'Pos {i}' for i in range(attention_weights.shape[1])],
                    yticklabels=[f'Pos {i}' for i in range(attention_weights.shape[0])])
        
        plt.title(title)
        plt.xlabel('Key Positions')
        plt.ylabel('Query Positions')
        plt.tight_layout()
        plt.show()
        
        return attention_weights
    
    def analyze_attention_patterns(self, attention_weights):
        """Analisa padrões de atenção"""
        
        # Converter para numpy
        if isinstance(attention_weights, torch.Tensor):
            attention_weights = attention_weights.detach().numpy()
        
        # Se for multi-head, usar a média
        if len(attention_weights.shape) == 4:
            attention_weights = np.mean(attention_weights, axis=1)
        
        # Remover dimensão do batch se existir
        if len(attention_weights.shape) == 3:
            attention_weights = attention_weights[0]
        
        # Análise dos padrões
        print(f"\n=== ANÁLISE DOS PADRÕES DE ATENÇÃO ===")
        
        # Entropia da atenção
        entropy = -np.sum(attention_weights * np.log(attention_weights + 1e-8), axis=1)
        print(f"Entropia média: {np.mean(entropy):.3f}")
        print(f"Entropia máxima: {np.max(entropy):.3f}")
        print(f"Entropia mínima: {np.min(entropy):.3f}")
        
        # Concentração da atenção
        max_attention = np.max(attention_weights, axis=1)
        print(f"\nConcentração máxima média: {np.mean(max_attention):.3f}")
        print(f"Posições com maior concentração: {np.argmax(max_attention)}")
        
        # Padrões locais vs globais
        diagonal_attention = np.diag(attention_weights)
        off_diagonal_attention = attention_weights - np.diag(diagonal_attention)
        
        print(f"\nAtenção diagonal média: {np.mean(diagonal_attention):.3f}")
        print(f"Atenção off-diagonal média: {np.mean(off_diagonal_attention):.3f}")
        
        return {
            'entropy': entropy,
            'max_attention': max_attention,
            'diagonal_attention': diagonal_attention,
            'off_diagonal_attention': off_diagonal_attention
        }

def demonstrate_attention_mechanisms():
    """Demonstra diferentes mecanismos de atenção"""
    
    print("=== Demonstração de Mecanismos de Atenção ===")
    
    # Criar instância
    demo = AttentionMechanismsDemo(seq_len=10, d_model=64)
    
    # Criar dados de exemplo
    print("\nCriando dados de exemplo...")
    x = demo.create_sample_data()
    
    # Self-attention
    print("\nExecutando self-attention...")
    sa_output, sa_weights = demo.self_attention(x)
    demo.visualize_attention_weights(sa_weights, "Self-Attention")
    sa_analysis = demo.analyze_attention_patterns(sa_weights)
    
    # Multi-head attention
    print("\nExecutando multi-head attention...")
    mha_output, mha_weights = demo.multi_head_attention(x, num_heads=8)
    demo.visualize_attention_weights(mha_weights, "Multi-Head Attention (Média)")
    mha_analysis = demo.analyze_attention_patterns(mha_weights)
    
    # Cross-attention
    print("\nExecutando cross-attention...")
    context = torch.randn(1, 8, 64)  # Contexto diferente
    ca_output, ca_weights = demo.cross_attention(x, context)
    demo.visualize_attention_weights(ca_weights, "Cross-Attention")
    ca_analysis = demo.analyze_attention_patterns(ca_weights)
    
    return sa_weights, mha_weights, ca_weights

# Executar demonstração
sa_weights, mha_weights, ca_weights = demonstrate_attention_mechanisms()

### Análise dos Resultados

**Mecanismos de Atenção Observados:**

1. **Self-Attention**: Cada posição interage com todas as outras
2. **Multi-Head Attention**: Múltiplas representações de atenção
3. **Cross-Attention**: Interação entre sequências diferentes
4. **Padrões**: Concentração em posições relevantes

**Insights Importantes:**
- **Foco seletivo**: Atenção concentrada em posições importantes
- **Contexto global**: Consideração de toda a sequência
- **Paralelização**: Processamento simultâneo de relações
- **Flexibilidade**: Adaptação a diferentes padrões

**Referências:**
- [Attention Is All You Need - Vaswani et al.](https://arxiv.org/abs/1706.03762)


## 8.3 Vision Transformers (ViT)**Vision Transformers** são uma adaptação da arquitetura Transformer para processamento de imagens, revolucionando a visão computacional ao aplicar mecanismos de atenção diretamente em patches de imagem.![Vision Transformers](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/vision_transformers.png)### Arquitetura Vision Transformer**Componentes Principais:****1. Patch Embedding:**- **Divisão em patches**: Imagem dividida em patches de 16×16 ou 32×32- **Embedding linear**: Cada patch convertido em vetor de dimensão fixa- **Posição**: Embedding de posição adicionado a cada patch- **CLS token**: Token especial para classificação**2. Transformer Encoder:**- **Multi-Head Attention**: Múltiplas cabeças de atenção- **Feed-Forward Networks**: Redes feed-forward- **Layer Normalization**: Normalização de camadas- **Residual Connections**: Conexões residuais![Arquitetura Vision Transformer](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/arquitetura_vision_transformer.png)**3. Classification Head:**- **CLS token**: Token de classificação- **MLP**: Multi-layer perceptron- **Softmax**: Probabilidades de classe- **Output**: Predição da classe### Processo de Processamento**1. Pré-processamento:**- **Redimensionamento**: Imagem redimensionada para tamanho fixo- **Divisão em patches**: Imagem dividida em patches- **Embedding**: Patches convertidos em vetores- **Posição**: Embedding de posição adicionado**2. Processamento:**- **Multi-Head Attention**: Atenção entre patches- **Feed-Forward**: Processamento feed-forward- **Residual**: Conexões residuais- **Normalization**: Normalização de camadas**3. Classificação:**- **CLS token**: Token de classificação- **MLP**: Multi-layer perceptron- **Softmax**: Probabilidades- **Predição**: Classe final### Vantagens dos Vision Transformers**1. Escalabilidade:**- **Dados**: Performance melhora com mais dados- **Modelo**: Arquitetura escalável- **Computação**: Eficiência computacional- **Paralelização**: Processamento paralelo**2. Flexibilidade:**- **Arquitetura**: Arquitetura uniforme- **Tarefas**: Adaptável a diferentes tarefas- **Domínios**: Funciona em diferentes domínios- **Integração**: Fácil integração com outros modelos**3. Interpretabilidade:**- **Atenção**: Pesos de atenção interpretáveis- **Patches**: Foco em patches específicos- **Visualização**: Visualização de atenção- **Análise**: Análise de padrões![Vantagens Vision Transformers](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/vantagens_vision_transformers.png)**Referências:**- [An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.](https://arxiv.org/abs/2010.11929)

## 8.4 Demonstração Prática: Vision Transformer Simples

Vamos implementar e visualizar um Vision Transformer simples:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import cv2

class SimpleVisionTransformer:
    """Implementação de um Vision Transformer simples para demonstração"""
    
    def __init__(self, img_size=64, patch_size=8, num_classes=10, d_model=128, num_heads=8, num_layers=4):
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_classes = num_classes
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Calcular número de patches
        self.num_patches = (img_size // patch_size) ** 2
        
        # Inicializar modelo
        self.model = self._build_vit()
        
    def _build_vit(self):
        """Constrói o Vision Transformer"""
        
        class VisionTransformer(nn.Module):
            def __init__(self, img_size, patch_size, num_classes, d_model, num_heads, num_layers, num_patches):
                super(VisionTransformer, self).__init__()
                
                self.img_size = img_size
                self.patch_size = patch_size
                self.num_classes = num_classes
                self.d_model = d_model
                self.num_heads = num_heads
                self.num_layers = num_layers
                self.num_patches = num_patches
                
                # Patch embedding
                self.patch_embedding = nn.Conv2d(3, d_model, kernel_size=patch_size, stride=patch_size)
                
                # Positional embedding
                self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, d_model))
                
                # CLS token
                self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
                
                # Transformer encoder
                encoder_layer = nn.TransformerEncoderLayer(
                    d_model=d_model,
                    nhead=num_heads,
                    dim_feedforward=d_model * 4,
                    dropout=0.1,
                    batch_first=True
                )
                self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
                
                # Classification head
                self.classification_head = nn.Sequential(
                    nn.LayerNorm(d_model),
                    nn.Linear(d_model, d_model // 2),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(d_model // 2, num_classes)
                )
                
            def forward(self, x):
                batch_size = x.shape[0]
                
                # Patch embedding
                x = self.patch_embedding(x)  # (batch, d_model, H/patch_size, W/patch_size)
                x = x.flatten(2).transpose(1, 2)  # (batch, num_patches, d_model)
                
                # Adicionar CLS token
                cls_tokens = self.cls_token.expand(batch_size, -1, -1)
                x = torch.cat((cls_tokens, x), dim=1)  # (batch, num_patches + 1, d_model)
                
                # Adicionar positional embedding
                x = x + self.pos_embedding
                
                # Transformer encoder
                x = self.transformer_encoder(x)
                
                # Classification head (usar apenas CLS token)
                cls_output = x[:, 0]  # (batch, d_model)
                output = self.classification_head(cls_output)
                
                return output, x
        
        return VisionTransformer(
            self.img_size, self.patch_size, self.num_classes,
            self.d_model, self.num_heads, self.num_layers, self.num_patches
        ).to(self.device)
    
    def create_sample_images(self, num_images=16):
        """Cria imagens de exemplo para demonstração"""
        
        images = []
        labels = []
        
        for i in range(num_images):
            # Criar imagem base
            img = np.zeros((self.img_size, self.img_size, 3), dtype=np.float32)
            
            # Adicionar padrões diferentes
            if i % 4 == 0:
                # Círculos
                cv2.circle(img, (32, 32), 20, (1, 0, 0), -1)
                label = 0
            elif i % 4 == 1:
                # Retângulos
                cv2.rectangle(img, (20, 20), (44, 44), (0, 1, 0), -1)
                label = 1
            elif i % 4 == 2:
                # Triângulos
                pts = np.array([[32, 20], [20, 44], [44, 44]], np.int32)
                cv2.fillPoly(img, [pts], (0, 0, 1))
                label = 2
            else:
                # Linhas
                cv2.line(img, (20, 20), (44, 44), (1, 1, 0), 3)
                label = 3
            
            # Adicionar ruído
            noise = np.random.normal(0, 0.1, img.shape)
            img = np.clip(img + noise, 0, 1)
            
            images.append(img)
            labels.append(label)
        
        return np.array(images), np.array(labels)
    
    def visualize_patches(self, image, patch_size=8):
        """Visualiza os patches de uma imagem"""
        
        # Converter para tensor
        if isinstance(image, np.ndarray):
            image = torch.FloatTensor(image).permute(2, 0, 1).unsqueeze(0)
        
        # Aplicar patch embedding
        patches = self.model.patch_embedding(image)
        patches = patches.flatten(2).transpose(1, 2)
        
        # Converter para numpy
        patches = patches.squeeze().detach().numpy()
        
        # Visualizar patches
        num_patches_per_side = self.img_size // patch_size
        
        fig, axes = plt.subplots(num_patches_per_side, num_patches_per_side, figsize=(12, 12))
        
        for i in range(num_patches_per_side):
            for j in range(num_patches_per_side):
                patch_idx = i * num_patches_per_side + j
                patch = patches[patch_idx]
                
                # Normalizar patch
                patch_normalized = (patch - patch.min()) / (patch.max() - patch.min() + 1e-8)
                
                # Visualizar como heatmap
                axes[i, j].imshow(patch_normalized.reshape(patch_size, patch_size), cmap='viridis')
                axes[i, j].set_title(f'Patch {patch_idx}')
                axes[i, j].axis('off')
        
        plt.suptitle('Visualização dos Patches', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return patches
    
    def visualize_attention(self, image, layer_idx=0):
        """Visualiza os pesos de atenção"""
        
        # Converter para tensor
        if isinstance(image, np.ndarray):
            image = torch.FloatTensor(image).permute(2, 0, 1).unsqueeze(0).to(self.device)
        
        # Forward pass
        self.model.eval()
        with torch.no_grad():
            output, features = self.model(image)
        
        # Simular pesos de atenção (em implementação real, extrair do modelo)
        num_patches = self.num_patches + 1  # +1 para CLS token
        attention_weights = torch.randn(num_patches, num_patches)
        attention_weights = F.softmax(attention_weights, dim=-1)
        
        # Visualizar atenção do CLS token
        cls_attention = attention_weights[0, 1:]  # Excluir CLS token
        cls_attention = cls_attention.reshape(self.img_size // self.patch_size, self.img_size // self.patch_size)
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Imagem original
        if isinstance(image, torch.Tensor):
            img_np = image.squeeze().permute(1, 2, 0).cpu().numpy()
        else:
            img_np = image
        
        axes[0].imshow(img_np)
        axes[0].set_title('Imagem Original')
        axes[0].axis('off')
        
        # Mapa de atenção
        im = axes[1].imshow(cls_attention.cpu().numpy(), cmap='hot', interpolation='nearest')
        axes[1].set_title('Mapa de Atenção (CLS Token)')
        axes[1].axis('off')
        
        # Adicionar colorbar
        plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
        
        plt.suptitle('Visualização da Atenção', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return cls_attention
    
    def analyze_model_performance(self, images, labels):
        """Analisa a performance do modelo"""
        
        # Converter para tensor
        images_tensor = torch.FloatTensor(images).permute(0, 3, 1, 2).to(self.device)
        labels_tensor = torch.LongTensor(labels).to(self.device)
        
        # Forward pass
        self.model.eval()
        with torch.no_grad():
            outputs, features = self.model(images_tensor)
            predictions = torch.argmax(outputs, dim=1)
            probabilities = F.softmax(outputs, dim=1)
        
        # Calcular métricas
        accuracy = (predictions == labels_tensor).float().mean().item()
        
        # Visualizar resultados
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        
        for i in range(8):
            row = i // 4
            col = i % 4
            
            # Imagem
            axes[row, col].imshow(images[i])
            
            # Título com predição
            pred_class = predictions[i].item()
            true_class = labels[i]
            confidence = probabilities[i, pred_class].item()
            
            title = f'Pred: {pred_class}, True: {true_class}\nConf: {confidence:.2f}'
            axes[row, col].set_title(title)
            axes[row, col].axis('off')
        
        plt.suptitle(f'Performance do Vision Transformer (Accuracy: {accuracy:.2f})', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Resumo das métricas
        print(f"\n=== RESUMO DA PERFORMANCE ===")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Número de patches: {self.num_patches}")
        print(f"Dimensão do modelo: {self.d_model}")
        print(f"Número de cabeças: {self.num_heads}")
        print(f"Número de camadas: {self.num_layers}")
        
        return {
            'accuracy': accuracy,
            'predictions': predictions.cpu().numpy(),
            'probabilities': probabilities.cpu().numpy()
        }

def demonstrate_vision_transformer():
    """Demonstra um Vision Transformer simples"""
    
    print("=== Demonstração de Vision Transformer ===")
    print("\nNota: Esta demonstração usa dados sintéticos para fins educacionais.")
    print("Em aplicações reais, use datasets reais de imagens.")
    
    # Criar instância do ViT
    vit = SimpleVisionTransformer(img_size=64, patch_size=8, num_classes=4, d_model=128, num_heads=8, num_layers=4)
    
    # Criar imagens de exemplo
    print("\nCriando imagens de exemplo...")
    images, labels = vit.create_sample_images(num_images=16)
    
    # Visualizar patches
    print("\nVisualizando patches...")
    patches = vit.visualize_patches(images[0])
    
    # Visualizar atenção
    print("\nVisualizando atenção...")
    attention = vit.visualize_attention(images[0])
    
    # Analisar performance
    print("\nAnalisando performance...")
    performance = vit.analyze_model_performance(images, labels)
    
    return vit, images, labels, performance

# Executar demonstração
vit_model, sample_images, sample_labels, model_performance = demonstrate_vision_transformer()

### Análise dos Resultados

**Vision Transformer Observado:**

1. **Patches**: Imagem dividida em patches de 8×8
2. **Atenção**: Mapa de atenção do CLS token
3. **Performance**: Accuracy na classificação
4. **Arquitetura**: Encoder com múltiplas camadas

**Insights Importantes:**
- **Patches**: Divisão eficiente da imagem
- **Atenção**: Foco em regiões relevantes
- **Escalabilidade**: Arquitetura escalável
- **Flexibilidade**: Adaptável a diferentes tarefas

**Referências:**
- [An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.](https://arxiv.org/abs/2010.11929)


## 8.5 Comparação: CNNs vs Vision Transformers### Diferenças Fundamentais**CNNs:**- **Indução**: Indução de bias local- **Convolução**: Operações locais- **Hierarquia**: Processamento hierárquico- **Eficiência**: Eficiente para dados limitados**Vision Transformers:**- **Atenção**: Atenção global- **Patches**: Processamento de patches- **Uniformidade**: Arquitetura uniforme- **Escalabilidade**: Eficiente para grandes datasets![Comparação CNNs vs Vision Transformers](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/comparacao_cnns_vit.png)### Vantagens e Desvantagens**CNNs - Vantagens:**- **Eficiência**: Eficiente para dados limitados- **Indução**: Bias local apropriado- **Maturidade**: Arquitetura madura- **Recursos**: Menos recursos computacionais**CNNs - Desvantagens:**- **Limitação**: Campo receptivo limitado- **Hierarquia**: Processamento hierárquico- **Flexibilidade**: Menos flexível- **Escalabilidade**: Limitada escalabilidade**Vision Transformers - Vantagens:**- **Escalabilidade**: Escalável com dados- **Atenção**: Atenção global- **Flexibilidade**: Arquitetura flexível- **Uniformidade**: Arquitetura uniforme**Vision Transformers - Desvantagens:**- **Dados**: Requer muitos dados- **Recursos**: Mais recursos computacionais- **Complexidade**: Mais complexo- **Indução**: Menos bias local### Aplicações Práticas**CNNs - Casos de Uso:**- **Dados limitados**: Quando há poucos dados- **Recursos limitados**: Quando há limitações computacionais- **Tarefas específicas**: Tarefas que se beneficiam de bias local- **Deploy**: Deploy em dispositivos com recursos limitados**Vision Transformers - Casos de Uso:**- **Grandes datasets**: Quando há muitos dados- **Recursos abundantes**: Quando há recursos computacionais- **Tarefas complexas**: Tarefas que requerem atenção global- **Pesquisa**: Pesquisa e desenvolvimento![Aplicações Práticas](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo8/aplicacoes_praticas.png)**Referências:**- [An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.](https://arxiv.org/abs/2010.11929)

## Resumo do Módulo 8

### Principais Conceitos Abordados

1. **Mecanismos de Atenção**
   - Conceitos fundamentais
   - Tipos de atenção
   - Evolução histórica

2. **Vision Transformers**
   - Arquitetura e componentes
   - Processo de processamento
   - Vantagens e limitações

3. **Implementação Prática**
   - Mecanismos de atenção
   - Vision Transformer simples
   - Visualização de resultados

4. **Comparação e Aplicações**
   - CNNs vs Vision Transformers
   - Vantagens e desvantagens
   - Casos de uso práticos

### Demonstrações Práticas

**1. Mecanismos de Atenção:**
   - Self-attention
   - Multi-head attention
   - Cross-attention
   - Visualização de pesos

**2. Vision Transformer:**
   - Patch embedding
   - Transformer encoder
   - Classification head
   - Visualização de atenção

### Próximos Passos

No próximo módulo, exploraremos **Foundation Models para Visão Computacional**, onde aprenderemos sobre modelos como CLIP, DALL-E e GPT-4V.

### Referências Principais

- [Attention Is All You Need - Vaswani et al.](https://arxiv.org/abs/1706.03762)
- [An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.](https://arxiv.org/abs/2010.11929)
- [CLIP: Learning Transferable Visual Representations - Radford et al.](https://arxiv.org/abs/2103.00020)

---

**Próximo Módulo**: Foundation Models para Visão Computacional
